<a href="https://colab.research.google.com/github/zhou100/FoodSecurityPrediction/blob/master/PredictFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use machine learning to predict food security

 ## Read in the data cleaned out using R

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
!cp /gdrive/My\ Drive/Colab\ Notebooks/fs_data/mw_dataset_hh.csv ./

In [0]:
!cp /gdrive/My\ Drive/Colab\ Notebooks/fs_data/mw_dataset_cluster.csv ./

In [0]:
!cp /gdrive/My\ Drive/Colab\ Notebooks/fs_data/tz_dataset_hh.csv ./

In [0]:
!cp /gdrive/My\ Drive/Colab\ Notebooks/fs_data/tz_dataset_cluster.csv ./

In [0]:
!cp /gdrive/My\ Drive/Colab\ Notebooks/fs_data/ug_dataset_hh.csv ./

In [0]:
!cp /gdrive/My\ Drive/Colab\ Notebooks/fs_data/ug_dataset_cluster.csv ./

## Preprocessing data


In [0]:
# Set up Notebook
% matplotlib inline

# Standard imports
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from matplotlib import cm


# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score

In [0]:
# Define function of split train and test data

In [0]:
y_train = train_data[labels]
y_test = test_data[labels]
y_test.head()

In [0]:
from sklearn.preprocessing import Imputer,StandardScaler

imp = Imputer(missing_values='NaN', strategy='mean', axis=0, copy=False)
x_train = imp.fit_transform(x_train)
x_test = imp.fit_transform(x_test)

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.fit_transform(x_test)

## 

## Apply algorithms

In [0]:
from sklearn.linear_model import LinearRegression

# Create and fit our linear regression model to training data
model = LinearRegression(fit_intercept=True)
model.fit(x_train, y_train[labels[2]])

# Compute model predictions for test data
pred_linear = model.predict(x_test)

actual_linear = y_test[labels[2]]
r2_linear = stats.pearsonr(actual_linear, pred_linear)[0] ** 2
r2_linear

In [0]:
from sklearn.ensemble import RandomForestRegressor

# Create Regressor with default properties
rfc = RandomForestRegressor(random_state =0,n_jobs =4,warm_start = True,max_depth=4, min_samples_leaf=5 )

rfc.fit(x_train, y_train[labels[2]])
# Fit estimator and display score

# Regress on test data
pred = rfc.predict(x_test)

actual = y_test[labels[2]]
r2_rfc = stats.pearsonr(actual, pred)[0] ** 2
r2_rfc

In [0]:
from sklearn.linear_model import RidgeCV
ridge = RidgeCV(alphas=(400,800), fit_intercept=True, normalize=False, scoring=None, cv=5, gcv_mode='auto', store_cv_values=False) 

# Define different alpha values for different fits
# alpha = [0.0, 1E-6, 1E-4, 1E-2, 1.0]

ridge.fit(x_train, y_train[labels[2]])
pred = ridge.predict(x_test)

actual = y_test[labels[2]]
r2_ridge= stats.pearsonr(actual, pred)[0] ** 2
r2_ridge

In [0]:
from sklearn.linear_model import LassoCV

ls = LassoCV(eps=0.001, n_alphas=100, alphas=(0.01, 2), fit_intercept=True,precompute='auto',n_jobs=4, random_state=0, selection='cyclic')

ls = ls.fit(x_train, y_train[labels[2]])
pred = ls.predict(x_test)

actual = y_test[labels[2]]
r2_ls= stats.pearsonr(actual, pred)[0] ** 2
r2_ls

In [0]:
from sklearn.linear_model import ElasticNetCV
en = ElasticNetCV(alphas=(0.1,0.02,3,2), copy_X=True, cv=10, eps=0.004, fit_intercept=True,
       l1_ratio=0.33, max_iter=1000, n_alphas=100, n_jobs=1,
       normalize=False, positive=False, precompute='auto', random_state=0,
       selection='cyclic', tol=0.0001, verbose=0)

en.fit(x_train, y_train[labels[2]])

pred_en = en.predict(x_test)

actual_en = y_test[labels[2]]
r2_en= stats.pearsonr(actual_en, pred_en)[0] ** 2
r2_en

## Visualization

In [0]:
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs[0, 0].scatter( actual_linear,pred_linear)
axs[0, 0].plot(actual_linear,actual_linear, color='red')

axs[1, 0].scatter( actual_rfr, pred_rfr)
axs[1, 0].plot(actual_rfr, actual_rfr, color='red')


axs[0, 1].scatter(actual_en ,pred_en )
axs[0, 1].plot(actual_en,actual_en, color='red')

axs[1, 1].scatter(actual_gbr, pred_gbr )
axs[1, 1].plot(actual_gbr, actual_gbr, color='red')


axs[0, 0].title.set_text('Scatter Plot Linear')
axs[1, 0].title.set_text('Scatter Plot RandomForest')
axs[0, 1].title.set_text('Scatter Plot ElasticNet')
axs[1, 1].title.set_text('Scatter Plot GradientBoosting')
plt.show()

## Interpretation
